In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess data
df2 = pd.read_excel("Features1D/Features.xlsx")
X2 = df2.drop(['mood'], axis=1)
y2 = df2['mood']

# Encode labels
le = LabelEncoder()
y2 = le.fit_transform(y2)

# Encode categorical features
X2['scale'] = le.fit_transform(X2['scale'])
X2['key'] = le.fit_transform(X2['key'])

# Split data
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.9, random_state=13, shuffle=True, stratify=y2)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_train, y2_train, train_size=0.8, random_state=13, shuffle=True, stratify=y2_train)

# Scale features
scaler = MinMaxScaler()
X2_train_scaled = scaler.fit_transform(X2_train.drop('file', axis=1))
X2_val_scaled = scaler.transform(X2_val.drop('file', axis=1))
X2_test_scaled = scaler.transform(X2_test.drop('file', axis=1))

# Convert to PyTorch tensors with correct shape
X2_train_tensor = torch.tensor(X2_train_scaled, dtype=torch.float32).unsqueeze(1)
y2_train_tensor = torch.tensor(y2_train, dtype=torch.long)
X2_val_tensor = torch.tensor(X2_val_scaled, dtype=torch.float32).unsqueeze(1)
y2_val_tensor = torch.tensor(y2_val, dtype=torch.long)
X2_test_tensor = torch.tensor(X2_test_scaled, dtype=torch.float32).unsqueeze(1)
y2_test_tensor = torch.tensor(y2_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X2_train_tensor, y2_train_tensor)
val_dataset = TensorDataset(X2_val_tensor, y2_val_tensor)
test_dataset = TensorDataset(X2_test_tensor, y2_test_tensor)

batchSize = 128
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define the model
class Conv1DModel(nn.Module):
    def __init__(self):
        super(Conv1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, padding='same')
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=7, padding='same')
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=7, padding='same')
        self.bn3 = nn.BatchNorm1d(32)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        
        self.flatten = nn.Flatten()
        
        # Adjust the number of units in the first fully connected layer based on the input shape
        self.fc1 = nn.Linear(32 * (X2_train_scaled.shape[1] // 8), 128)
        self.dropout1 = nn.Dropout(0.7)
        
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.6)
        
        self.fc3 = nn.Linear(64, 5)
        
    def forward(self, x):
        x = self.pool1(self.bn1(torch.relu(self.conv1(x))))
        x = self.pool2(self.bn2(torch.relu(self.conv2(x))))
        x = self.pool3(self.bn3(torch.relu(self.conv3(x))))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.softmax(self.fc3(x), dim=1)
        return x

# Instantiate the model and move to GPU
model = Conv1DModel().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-2)

# Training loop
num_epochs = 2000
best_val_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_accuracy += (predicted == y_batch).sum().item()

    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            val_outputs = model(X_val_batch)
            val_loss += criterion(val_outputs, y_val_batch).item()
            _, predicted = torch.max(val_outputs, 1)
            val_accuracy += (predicted == y_val_batch).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy /= len(val_loader.dataset)

    # Update best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
              f'Best Val Accuracy: {best_val_accuracy:.4f}')


Using device: cuda
Epoch [100/2000], Train Loss: 1.1798, Train Accuracy: 0.8194, Val Loss: 1.1205, Val Accuracy: 0.8556, Best Val Accuracy: 0.8556
Epoch [200/2000], Train Loss: 0.9613, Train Accuracy: 0.9833, Val Loss: 1.0191, Val Accuracy: 0.8778, Best Val Accuracy: 0.8889
Epoch [300/2000], Train Loss: 0.9423, Train Accuracy: 0.9861, Val Loss: 1.0214, Val Accuracy: 0.8778, Best Val Accuracy: 0.9000
Epoch [400/2000], Train Loss: 0.9368, Train Accuracy: 0.9917, Val Loss: 1.0162, Val Accuracy: 0.8778, Best Val Accuracy: 0.9000
Epoch [500/2000], Train Loss: 0.9246, Train Accuracy: 0.9972, Val Loss: 1.0260, Val Accuracy: 0.8778, Best Val Accuracy: 0.9111
Epoch [600/2000], Train Loss: 0.9255, Train Accuracy: 1.0000, Val Loss: 1.0316, Val Accuracy: 0.8667, Best Val Accuracy: 0.9111
Epoch [700/2000], Train Loss: 0.9202, Train Accuracy: 1.0000, Val Loss: 1.0347, Val Accuracy: 0.8778, Best Val Accuracy: 0.9111
Epoch [800/2000], Train Loss: 0.9173, Train Accuracy: 1.0000, Val Loss: 1.0326, Val A

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess data
df2 = pd.read_excel("Features1D/Features.xlsx")
X2 = df2.drop(['mood'], axis=1)
y2 = df2['mood']

# Encode labels
le = LabelEncoder()
y2 = le.fit_transform(y2)

# Encode categorical features
X2['scale'] = le.fit_transform(X2['scale'])
X2['key'] = le.fit_transform(X2['key'])

# Split data
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.9, random_state=13, shuffle=True, stratify=y2)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_train, y2_train, train_size=0.8, random_state=13, shuffle=True, stratify=y2_train)

# Scale features
scaler = MinMaxScaler()
X2_train_scaled = scaler.fit_transform(X2_train.drop('file', axis=1))
X2_val_scaled = scaler.transform(X2_val.drop('file', axis=1))
X2_test_scaled = scaler.transform(X2_test.drop('file', axis=1))

# Convert to PyTorch tensors with correct shape
X2_train_tensor = torch.tensor(X2_train_scaled, dtype=torch.float32).unsqueeze(1)
y2_train_tensor = torch.tensor(y2_train, dtype=torch.long)
X2_val_tensor = torch.tensor(X2_val_scaled, dtype=torch.float32).unsqueeze(1)
y2_val_tensor = torch.tensor(y2_val, dtype=torch.long)
X2_test_tensor = torch.tensor(X2_test_scaled, dtype=torch.float32).unsqueeze(1)
y2_test_tensor = torch.tensor(y2_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X2_train_tensor, y2_train_tensor)
val_dataset = TensorDataset(X2_val_tensor, y2_val_tensor)
test_dataset = TensorDataset(X2_test_tensor, y2_test_tensor)

batchSize = 128
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define the model
class Conv1DModel(nn.Module):
    def __init__(self):
        super(Conv1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, padding='same')
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=7, padding='same')
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=7, padding='same')
        self.bn3 = nn.BatchNorm1d(32)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        
        self.flatten = nn.Flatten()
        
        # Adjust the number of units in the first fully connected layer based on the input shape
        self.fc1 = nn.Linear(32 * (X2_train_scaled.shape[1] // 8), 128)
        self.dropout1 = nn.Dropout(0.6)
        
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.6)
        
        self.fc3 = nn.Linear(64, 5)
        
    def forward(self, x):
        x = self.pool1(self.bn1(torch.relu(self.conv1(x))))
        x = self.pool2(self.bn2(torch.relu(self.conv2(x))))
        x = self.pool3(self.bn3(torch.relu(self.conv3(x))))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.softmax(self.fc3(x), dim=1)
        return x

# Instantiate the model and move to GPU
model = Conv1DModel().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-2)

# Training loop
num_epochs = 2000
best_val_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_accuracy += (predicted == y_batch).sum().item()

    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            val_outputs = model(X_val_batch)
            val_loss += criterion(val_outputs, y_val_batch).item()
            _, predicted = torch.max(val_outputs, 1)
            val_accuracy += (predicted == y_val_batch).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy /= len(val_loader.dataset)

    # Update best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
              f'Best Val Accuracy: {best_val_accuracy:.4f}')


Using device: cuda
Epoch [100/2000], Train Loss: 1.1328, Train Accuracy: 0.8472, Val Loss: 1.1486, Val Accuracy: 0.8556, Best Val Accuracy: 0.8556
Epoch [200/2000], Train Loss: 0.9453, Train Accuracy: 0.9861, Val Loss: 1.0122, Val Accuracy: 0.9000, Best Val Accuracy: 0.9111
Epoch [300/2000], Train Loss: 0.9303, Train Accuracy: 0.9889, Val Loss: 1.0168, Val Accuracy: 0.9000, Best Val Accuracy: 0.9111
Epoch [400/2000], Train Loss: 0.9235, Train Accuracy: 0.9972, Val Loss: 1.0227, Val Accuracy: 0.8889, Best Val Accuracy: 0.9111
Epoch [500/2000], Train Loss: 0.9243, Train Accuracy: 0.9972, Val Loss: 1.0249, Val Accuracy: 0.8889, Best Val Accuracy: 0.9111
Epoch [600/2000], Train Loss: 0.9187, Train Accuracy: 1.0000, Val Loss: 1.0254, Val Accuracy: 0.9111, Best Val Accuracy: 0.9222
Epoch [700/2000], Train Loss: 0.9255, Train Accuracy: 0.9972, Val Loss: 1.0328, Val Accuracy: 0.8889, Best Val Accuracy: 0.9222
Epoch [800/2000], Train Loss: 0.9194, Train Accuracy: 1.0000, Val Loss: 1.0277, Val A